In [1]:
import os
import sys
import logging
import configparser
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
import time
from datetime import datetime

# Get parent directory (project root)
project_root = Path.cwd().parent
sys.path.append(str(project_root))
os.chdir(project_root)
print(f"Project root directory: {project_root}")

Project root directory: /Users/giandomenico/Documents/SAPIENZA/Python Scripts/rockfall_risk_roads


In [2]:
"""
Minimal example showing how to integrate the intrinsic value module
with network relevance analysis for exposure assessment.
"""

import geopandas as gpd
from src.road.network_analysis import RoadNetworkGraph
from src.exposure.network_relevance import NetworkRelevanceAnalyzer, calculate_network_exposure
from src.exposure.intrinsic_value import IntrinsicValueCalculator
from src.utils.geo_utils import multilinestring_to_linestring


# Load road network
roads_path = 'data/input/tr_str.gpkg'
# Segmented roads path
roads_segmented_path = 'data/intermediate/road_segments.gpkg'

# roads = gpd.read_file(roads_path)
roads = gpd.read_file(roads_segmented_path)

# Trasform multiline to simple line string
if (roads.geometry.geom_type == 'MultiLineString').any():
    print("Converting MultiLineString to LineString...")
    road_segments = multilinestring_to_linestring(roads)
else:
    road_segments = roads

# Step 1: Calculate intrinsic values
intrinsic_calc = IntrinsicValueCalculator()
road_segments = intrinsic_calc.calculate_for_dataframe(road_segments)

# Step 2: Create network graph
road_network = RoadNetworkGraph()
graph = road_network.create_graph_from_segments(road_segments, segment_id_col='segment_id',)

# Step 3: Analyze network relevance
relevance_analyzer = NetworkRelevanceAnalyzer(graph, road_network)
relevance_analyzer.calculate_baseline_metrics()
df_impacts = relevance_analyzer.analyze_all_segments()
df_classified = relevance_analyzer.classify_segments_by_relevance(df_impacts)

# Step 4: Merge results back to GeoDataFrame
gdf_with_relevance = relevance_analyzer.export_relevance_to_geodataframe(road_segments, df_classified, segment_id_col='segment_id')

# Step 5: Calculate integrated exposure
gdf_final = calculate_network_exposure(
    gdf_with_relevance,
    relevance_weight=0.6,  # 60% network importance
    intrinsic_weight=0.4   # 40% physical characteristics
)


# Print summary
print(f"Total segments: {len(gdf_final)}")
print(f"High/Very High exposure: {len(gdf_final[gdf_final['exposure_class'].isin(['High', 'Very High'])])}")
print("\nExposure distribution:")
print(gdf_final['exposure_class'].value_counts())

Total segments: 594
High/Very High exposure: 131

Exposure distribution:
Low          232
Medium       231
High         104
Very High     27
Very Low       0
Name: exposure_class, dtype: int64


In [3]:
from src.utils.io_utils import write_vector
write_vector(gdf_final, 'data/output/road_segments_exposure_assessment.gpkg')

In [4]:
from src.exposure.exposure_utils import create_interactive_exposure_map
import folium

# Create interactive map
create_interactive_exposure_map(gdf_final, output_file='data/output/road_segments_exposure_map.html',focus_on_critical=False)

Interactive map saved to: data/output/road_segments_exposure_map.html
